# Plastic Waste Surveillance System

Municipal waste management platform with automated detection and analytics

In [ ]:
%%capture
!pip install ultralytics gradio plotly opencv-python-headless numpy pillow
print('Installation complete')

In [ ]:
import cv2
import numpy as np
import gradio as gr
from PIL import Image
from ultralytics import YOLO
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
import math
from collections import defaultdict

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
model = YOLO('yolov8m.pt')

PLASTIC_CATEGORIES = {
    'bottle': {'name': 'PET Bottle', 'hazard': 'Medium', 'recyclable': True},
    'cup': {'name': 'Plastic Cup', 'hazard': 'Low', 'recyclable': False},
    'bowl': {'name': 'Container', 'hazard': 'Low', 'recyclable': True},
    'fork': {'name': 'Cutlery', 'hazard': 'Low', 'recyclable': False},
    'knife': {'name': 'Cutlery', 'hazard': 'Low', 'recyclable': False},
    'spoon': {'name': 'Cutlery', 'hazard': 'Low', 'recyclable': False},
    'cell phone': {'name': 'E-Waste', 'hazard': 'High', 'recyclable': True},
    'laptop': {'name': 'E-Waste', 'hazard': 'High', 'recyclable': True},
    'backpack': {'name': 'Plastic Bag', 'hazard': 'Medium', 'recyclable': False},
    'handbag': {'name': 'Plastic Bag', 'hazard': 'Medium', 'recyclable': False},
}

detection_database = []

In [ ]:
def calculate_location(x, y, width, height):
    center_x, center_y = width / 2, height / 2
    dx, dy = x - center_x, y - center_y
    distance = math.sqrt(dx**2 + dy**2)
    distance_feet = (distance / max(width, height)) * 10
    angle = math.degrees(math.atan2(dy, dx))

    if -22.5 <= angle < 22.5: direction = 'East'
    elif 22.5 <= angle < 67.5: direction = 'Southeast'
    elif 67.5 <= angle < 112.5: direction = 'South'
    elif 112.5 <= angle < 157.5: direction = 'Southwest'
    elif angle >= 157.5 or angle < -157.5: direction = 'West'
    elif -157.5 <= angle < -112.5: direction = 'Northwest'
    elif -112.5 <= angle < -67.5: direction = 'North'
    else: direction = 'Northeast'

    sector_x, sector_y = int((x / width) * 3), int((y / height) * 3)
    return {'distance': round(distance_feet, 1), 'direction': direction, 'sector': f'S{sector_y}{sector_x}'}

def detect_small_plastic(img):
    h, w = img.shape[:2]
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    detections = []

    color_ranges = [
        {'lower': np.array([0, 100, 100]), 'upper': np.array([15, 255, 255])},
        {'lower': np.array([90, 80, 80]), 'upper': np.array([130, 255, 255])},
        {'lower': np.array([40, 80, 80]), 'upper': np.array([80, 255, 255])},
        {'lower': np.array([20, 100, 100]), 'upper': np.array([35, 255, 255])},
        {'lower': np.array([0, 0, 180]), 'upper': np.array([180, 40, 255])},
    ]

    for color_range in color_ranges:
        mask = cv2.inRange(hsv, color_range['lower'], color_range['upper'])
        kernel = np.ones((3, 3), np.uint8)
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        for cnt in contours:
            area = cv2.contourArea(cnt)
            if 200 < area < 15000:
                x, y, bw, bh = cv2.boundingRect(cnt)
                conf = min(0.5 + (area / 5000) * 0.2, 0.85)
                detections.append({'bbox': (x, y, x+bw, y+bh), 'confidence': conf, 'type': 'Plastic Item', 'method': 'Color'})
    return detections[:8]

def comprehensive_detect(image, conf_threshold=0.35):
    if isinstance(image, Image.Image):
        img = np.array(image)
    else:
        img = image

    h, w = img.shape[:2]
    results = model.predict(img, conf=conf_threshold, iou=0.4, verbose=False)
    detections = []

    for result in results:
        if result.boxes is None:
            continue
        for box in result.boxes:
            cls_name = model.names[int(box.cls[0])].lower()
            if cls_name not in PLASTIC_CATEGORIES:
                continue

            conf = float(box.conf[0])
            x1, y1, x2, y2 = map(int, box.xyxy[0].cpu().numpy())
            cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
            cat = PLASTIC_CATEGORIES[cls_name]

            detections.append({
                'type': cat['name'], 'confidence': conf, 'bbox': (x1, y1, x2, y2),
                'hazard': cat['hazard'], 'recyclable': cat['recyclable'],
                'location': calculate_location(cx, cy, w, h), 'method': 'YOLO', 'timestamp': datetime.now()
            })

    color_dets = detect_small_plastic(img)
    for det in color_dets:
        x1, y1, x2, y2 = det['bbox']
        det['location'] = calculate_location((x1+x2)//2, (y1+y2)//2, w, h)
        det['hazard'], det['recyclable'], det['timestamp'] = 'Low', False, datetime.now()

    final = remove_duplicates(detections + color_dets)
    annotated = img.copy()

    for det in final:
        x1, y1, x2, y2 = det['bbox']
        color = (0, 77, 64) if det['hazard'] == 'High' else (0, 105, 92) if det['hazard'] == 'Medium' else (0, 150, 136)
        cv2.rectangle(annotated, (x1, y1), (x2, y2), color, 3)
        label = f"{det['type']} {int(det['confidence']*100)}%"
        (tw, th), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
        cv2.rectangle(annotated, (x1, y1-th-8), (x1+tw+8, y1), color, -1)
        cv2.putText(annotated, label, (x1+4, y1-4), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 2)

    return annotated, final

def remove_duplicates(dets):
    if not dets: return []
    dets.sort(key=lambda x: x['confidence'], reverse=True)
    final = []
    for det in dets:
        x1, y1, x2, y2 = det['bbox']
        overlap = False
        for ex in final:
            ex1, ey1, ex2, ey2 = ex['bbox']
            ix1, iy1, ix2, iy2 = max(x1, ex1), max(y1, ey1), min(x2, ex2), min(y2, ey2)
            if ix2 > ix1 and iy2 > iy1:
                inter = (ix2-ix1) * (iy2-iy1)
                iou = inter / ((x2-x1)*(y2-y1) + (ex2-ex1)*(ey2-ey1) - inter)
                if iou > 0.3:
                    overlap = True
                    break
        if not overlap:
            final.append(det)
    return final

In [ ]:
def predict_hotspots(detections):
    if len(detections) < 2: return []
    sectors = defaultdict(list)
    for det in detections:
        sectors[det['location']['sector']].append(det)

    hotspots = []
    for sector, items in sectors.items():
        if len(items) >= 2:
            avg_dist = sum(i['location']['distance'] for i in items) / len(items)
            risk = 'CRITICAL' if len(items) >= 5 else 'HIGH' if len(items) >= 3 else 'MEDIUM'
            hotspots.append({
                'sector': sector, 'count': len(items), 'risk': risk,
                'avg_distance': round(avg_dist, 1), 'growth_rate': round(len(items) * 0.3, 1)
            })
    hotspots.sort(key=lambda x: x['count'], reverse=True)
    return hotspots

def generate_alert(detections, hotspots):
    total = len(detections)
    if total == 0:
        return {'level': 'CLEAR', 'status': 'No waste detected', 'action': 'Continue monitoring', 'total': 0, 'high_hazard': 0, 'hotspots': 0}

    high_hazard = sum(1 for d in detections if d.get('hazard') == 'High')

    if total >= 10 or high_hazard >= 3:
        level, status, action = 'CRITICAL', 'Immediate action required', 'Deploy cleanup crew within 1 hour'
    elif total >= 5 or high_hazard >= 1:
        level, status, action = 'HIGH', 'Action required', 'Schedule cleanup within 4 hours'
    elif total >= 2:
        level, status, action = 'MEDIUM', 'Monitoring required', 'Add to next cleanup schedule'
    else:
        level, status, action = 'LOW', 'Minor detection', 'Continue monitoring'

    return {'level': level, 'status': status, 'action': action, 'total': total, 'high_hazard': high_hazard, 'hotspots': len(hotspots)}

In [ ]:
def create_dashboard(detections, hotspots, alert):
    if not detections:
        fig = go.Figure()
        fig.add_annotation(text='No waste detected', x=0.5, y=0.5, showarrow=False, font={'size': 18, 'color': '#004d40'})
        fig.update_layout(height=550, xaxis={'visible': False}, yaxis={'visible': False}, paper_bgcolor='#fafafa')
        return fig

    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Waste Distribution', 'Hotspot Analysis', 'Hazard Assessment', 'Spatial Mapping'),
        specs=[[{'type': 'pie'}, {'type': 'bar'}], [{'type': 'bar'}, {'type': 'scatter'}]]
    )

    types = {}
    for d in detections:
        types[d['type']] = types.get(d['type'], 0) + 1

    fig.add_trace(go.Pie(labels=list(types.keys()), values=list(types.values()),
                         marker=dict(colors=['#004d40', '#00695c', '#00796b', '#00897b', '#26a69a'])), row=1, col=1)

    if hotspots:
        colors = ['#004d40' if h['risk']=='CRITICAL' else '#00695c' if h['risk']=='HIGH' else '#00897b' for h in hotspots]
        fig.add_trace(go.Bar(x=[h['sector'] for h in hotspots], y=[h['count'] for h in hotspots], marker_color=colors), row=1, col=2)

    hazards = {'High': 0, 'Medium': 0, 'Low': 0}
    for d in detections:
        hazards[d.get('hazard', 'Low')] += 1

    fig.add_trace(go.Bar(x=list(hazards.keys()), y=list(hazards.values()),
                         marker_color=['#004d40', '#00695c', '#26a69a']), row=2, col=1)

    distances = [d['location']['distance'] for d in detections]
    confidences = [d['confidence']*100 for d in detections]
    fig.add_trace(go.Scatter(x=distances, y=confidences, mode='markers',
                            marker={'size': 14, 'color': '#00796b', 'line': {'width': 2, 'color': '#fff'}}), row=2, col=2)

    fig.update_layout(height=750, showlegend=False, title_text=f"Analytics Dashboard - Alert: {alert['level']}",
                     paper_bgcolor='#ffffff', font={'family': 'Inter, Georgia, serif', 'color': '#263238'})
    fig.update_xaxes(title_text='Distance (feet)', row=2, col=2)
    fig.update_yaxes(title_text='Confidence (%)', row=2, col=2)
    return fig

In [ ]:
def process_surveillance(image, confidence):
    if image is None:
        return None, 'Please upload an image to begin analysis', None, '', '', ''

    annotated, dets = comprehensive_detect(image, confidence)
    detection_database.extend(dets)
    hotspots = predict_hotspots(dets)
    alert = generate_alert(dets, hotspots)

    report = f'''SURVEILLANCE REPORT\n{'='*60}\n\nTimestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\nMonitoring Radius: 10 feet\n\nDETECTION SUMMARY\nTotal Items: {len(dets)}\nAlert Level: {alert['level']}\nStatus: {alert['status']}\n\n'''

    if dets:
        report += 'DETECTED ITEMS\n' + '-'*60 + '\n'
        for i, d in enumerate(dets, 1):
            report += f"{i}. {d['type']} - Confidence: {d['confidence']*100:.0f}%\n   Location: Sector {d['location']['sector']}, {d['location']['direction']}, {d['location']['distance']} feet\n   Hazard Level: {d.get('hazard', 'Low')}\n\n"

    if hotspots:
        report += 'HOTSPOT ANALYSIS\n' + '-'*60 + '\n'
        for h in hotspots:
            report += f"Sector {h['sector']}: {h['count']} items detected, Risk: {h['risk']}, Growth: {h['growth_rate']} items/hour\n"

    report += f"\nRECOMMENDED ACTION\n{alert['action']}"

    alert_msg = f'''MUNICIPAL AUTHORITY ALERT\n{'='*50}\n\nAlert Level: {alert['level']}\nStatus: {alert['status']}\n\nDetection Statistics\nTotal Items: {alert['total']}\nHigh Hazard Items: {alert['high_hazard']}\nHotspots Identified: {alert['hotspots']}\n\nRecommended Action\n{alert['action']}'''

    summary = f"{alert['level']}: {len(dets)} items detected, {len(hotspots)} hotspots" if dets else 'Area Clear'
    stats = f"Total: {len(dets)} | AI Detection: {sum(1 for d in dets if d.get('method')=='YOLO')} | Color Analysis: {sum(1 for d in dets if d.get('method')=='Color')}"
    dashboard = create_dashboard(dets, hotspots, alert)

    return annotated, report, dashboard, alert_msg, summary, stats

In [ ]:
custom_css = '''
.gradio-container {
    font-family: 'Inter', 'Georgia', serif;
    max-width: 1400px;
    margin: 0 auto;
}
h1, h2, h3 { font-weight: 600; color: #00695c; letter-spacing: -0.02em; }
.primary-btn { background: #004d40 !important; color: white !important; }
'''

with gr.Blocks(css=custom_css, title='Waste Surveillance Platform', theme=gr.themes.Base(
    primary_hue='teal', secondary_hue='slate', neutral_hue='slate',
    font=['Inter', 'Georgia', 'serif']
)) as demo:

    gr.HTML('''
    <div style='background: linear-gradient(135deg, #004d40 0%, #00695c 50%, #00796b 100%); padding: 36px; text-align: center; border-radius: 8px; margin-bottom: 32px;'>
        <h1 style='color: white; margin: 0 0 10px 0; font-size: 34px; font-weight: 600; letter-spacing: -0.5px;'>Plastic Waste Surveillance System</h1>
    </div>
    ''')

    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown('<h3 style="color: #00695c; font-weight: 600;">Image Upload</h3>')
            input_img = gr.Image(label='Surveillance Feed', type='pil', height=260)

            gr.Markdown('<h3 style="color: #00695c; font-weight: 600;">Configuration</h3>')
            conf_slider = gr.Slider(0.25, 0.7, 0.35, 0.05, label='Detection Threshold', info='Adjust sensitivity')

            detect_btn = gr.Button('Run Analysis', variant='primary', size='lg', elem_classes='primary-btn')

            gr.Markdown('<h3 style="color: #00695c; font-weight: 600;">System Status</h3>')
            status_box = gr.Textbox(label='Alert Level', lines=1, interactive=False)
            stats_box = gr.Textbox(label='Detection Statistics', lines=1, interactive=False)

        with gr.Column(scale=1):
            gr.Markdown('<h3 style="color: #00695c; font-weight: 600;">Detection Results</h3>')
            output_img = gr.Image(label='Annotated Image', height=260)

            gr.Markdown('<h3 style="color: #00695c; font-weight: 600;">Authority Alert</h3>')
            alert_box = gr.Textbox(label='Municipal Notification', lines=9, interactive=False)

    gr.Markdown('<h3 style="color: #00695c; font-weight: 600;">Comprehensive Analysis</h3>')

    with gr.Row():
        report_box = gr.Textbox(label='Detailed Report', lines=13, interactive=False)
        dashboard_plot = gr.Plot(label='Analytics Dashboard')

    gr.HTML('''
    <div style='background: linear-gradient(to bottom, #fafafa 0%, #f5f5f5 100%); padding: 32px; border-radius: 8px; margin-top: 32px; border: 1px solid #e0e0e0;'>
        <h3 style='margin: 0 0 24px 0; color: #00695c; font-size: 22px; font-weight: 600;'>System Capabilities</h3>

        <div style='display: grid; grid-template-columns: repeat(3, 1fr); gap: 20px; margin-bottom: 24px;'>
            <div style='background: white; padding: 24px; border-radius: 6px; border-left: 4px solid #004d40;'>
                <div style='color: #004d40; font-weight: 600; font-size: 16px; margin-bottom: 10px;'>Dual Detection</div>
                <div style='color: #546e7a; font-size: 14px; line-height: 1.6;'>YOLOv8 neural network combined with color-based analysis for comprehensive coverage</div>
            </div>

            <div style='background: white; padding: 24px; border-radius: 6px; border-left: 4px solid #00796b;'>
                <div style='color: #00796b; font-weight: 600; font-size: 16px; margin-bottom: 10px;'>Spatial Tracking</div>
                <div style='color: #546e7a; font-size: 14px; line-height: 1.6;'>10-foot radius monitoring with precise location mapping and directional analysis</div>
            </div>

            <div style='background: white; padding: 24px; border-radius: 6px; border-left: 4px solid #26a69a;'>
                <div style='color: #26a69a; font-weight: 600; font-size: 16px; margin-bottom: 10px;'>Predictive Analytics</div>
                <div style='color: #546e7a; font-size: 14px; line-height: 1.6;'>Hotspot identification with waste accumulation forecasting and growth modeling</div>
            </div>
        </div>

        <div style='background: white; padding: 20px 24px; border-radius: 6px; border: 1px solid #e0e0e0;'>
            <span style='color: #263238; font-weight: 600; font-size: 15px;'>Detection Color Legend:</span>
            <span style='margin-left: 20px; color: #546e7a; font-size: 14px;'>
                <span style='display: inline-block; width: 14px; height: 14px; background: #004d40; border-radius: 3px; margin: 0 6px;'></span>High Hazard
                <span style='display: inline-block; width: 14px; height: 14px; background: #00695c; border-radius: 3px; margin: 0 6px 0 16px;'></span>Medium Hazard
                <span style='display: inline-block; width: 14px; height: 14px; background: #26a69a; border-radius: 3px; margin: 0 6px 0 16px;'></span>Low Hazard
            </span>
        </div>
    </div>
    ''')

    detect_btn.click(process_surveillance, [input_img, conf_slider],
                    [output_img, report_box, dashboard_plot, alert_box, status_box, stats_box])

demo.launch(share=True, show_error=True)

/tmp/ipython-input-1177381683.py:11: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(css=custom_css, title='Waste Surveillance Platform', theme=gr.themes.Base(
/tmp/ipython-input-1177381683.py:11: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(css=custom_css, title='Waste Surveillance Platform', theme=gr.themes.Base(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6052de399dac2cad88.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
